<a href="https://colab.research.google.com/github/deivismartinez/DDE/blob/main/NER2024_SS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.39  #span_marker use model.predic whit this tranformers version

In [ ]:
!pip install span_marker

In [ ]:
!pip install https://huggingface.co/spacy/es_core_news_md/resolve/main/es_core_news_md-any-py3-none-any.whl  #para spacy

In [ ]:
!pip install https://huggingface.co/PlanTL-GOB-ES/ca_anonimization_core_lg/resolve/main/ca_anonimization_core_lg-any-py3-none-any.whl

In [ ]:
!pip install https://huggingface.co/spacy/es_core_news_sm/resolve/main/es_core_news_sm-any-py3-none-any.whl

In [ ]:
!pip install https://huggingface.co/spacy/es_core_news_lg/resolve/main/es_core_news_lg-any-py3-none-any.whl

In [ ]:
import es_core_news_md
import ca_anonimization_core_lg
import es_core_news_sm
import es_core_news_lg

In [ ]:
!pip install stanza
!pip install typing_extensions==4.7.1 --upgrade

In [ ]:
import stanza
stanza.download("es")

In [ ]:
!pip install --user flair
from flair.data import Sentence
from flair.models import SequenceTagger

In [ ]:
!pip install torch==1.13.1 #Usar esta versión para flair fast
#en google colab se reinicia sesion y luego continuamos

     ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/887.5 MB 93.3 MB/s eta 0:00:08
ERROR: Operation cancelled by user


In [ ]:
!pip install gliner

In [ ]:
from gliner import GLiNER

In [ ]:
import json
from pathlib import Path
from google.colab import files
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
from math import ceil
import shutil
import os
import zipfile
import glob
import sys
import spacy

In [ ]:
!wget https://zenodo.org/record/4279323/files/meddocan.zip?download=1 -O meddocan.zip

--2024-07-18 20:42:18--  https://zenodo.org/record/4279323/files/meddocan.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4279323/files/meddocan.zip [following]
--2024-07-18 20:42:19--  https://zenodo.org/records/4279323/files/meddocan.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 11738792 (11M) [application/octet-stream]
Saving to: ‘meddocan.zip’

meddocan.zip        100%[===================>]  11.19M  4.34MB/s    in 2.6s    

2024-07-18 20:42:22 (4.34 MB/s) - ‘meddocan.zip’ saved [11738792/11738792]



In [ ]:
!unzip meddocan.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: meddocan/test/brat/S0376-78922015000100011-1.ann  
  inflating: meddocan/test/brat/S0376-78922015000100011-1.txt  
  inflating: meddocan/test/brat/S0376-78922016000300005-1.ann  
  inflating: meddocan/test/brat/S0376-78922016000300005-1.txt  
  inflating: meddocan/test/brat/S0376-78922016000300010-1.ann  
  inflating: meddocan/test/brat/S0376-78922016000300010-1.txt  
  inflating: meddocan/test/brat/S0378-48352004000300007-1.ann  
  inflating: meddocan/test/brat/S0378-48352004000300007-1.txt  
  inflating: meddocan/test/brat/S0378-48352005000100005-1.ann  
  inflating: meddocan/test/brat/S0378-48352005000100005-1.txt  
  inflating: meddocan/test/brat/S0378-48352006000300005-1.ann  
  inflating: meddocan/test/brat/S0378-48352006000300005-1.txt  
  inflating: meddocan/test/brat/S0378-48352006000400006-1.ann  
  inflating: meddocan/test/brat/S0378-48352006000400006-1.txt  
  inflating: meddocan/test/brat/S0378-48352

In [ ]:
dataset = {}
for path in (Path('dev'), Path('test'), Path('train')):
    dir_path = Path('meddocan')/path/Path('brat')
    filenames = tuple(f[:-4] for f in listdir(dir_path) if isfile(join(dir_path, f)) if f[-4:] == '.txt')
    dataset[str(path)] = []
    for file_name in filenames:
      d = dict()
      with open(dir_path/Path(file_name+'.txt'), 'r') as f:
        dataset[str(path)].append({"text":f.read(),"file":file_name,"file_name_path":dir_path/Path(file_name+'.txt')})

In [ ]:
!wget https://raw.githubusercontent.com/deivismartinez/NER-Medical-Uninorte/main/tag.json
!wget https://raw.githubusercontent.com/deivismartinez/NER-Medical-Uninorte/main/models.json

--2024-07-18 20:42:25--  https://raw.githubusercontent.com/deivismartinez/NER-Medical-Uninorte/main/tag.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102 [text/plain]
Saving to: ‘tag.json’

tag.json            100%[===================>]     102  --.-KB/s    in 0s      

2024-07-18 20:42:25 (658 KB/s) - ‘tag.json’ saved [102/102]

--2024-07-18 20:42:25--  https://raw.githubusercontent.com/deivismartinez/NER-Medical-Uninorte/main/models.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1903 (1.9K) [text/plain]
Saving to: ‘

In [ ]:
def get_models():
  #file = open("models.json","r")
  #models = json.load(file)
  #file.close()
  models = [
    {"id":1,"name":"roberta-large-NER", "folder":"51la5/"},
    {"id":2,"name":"bert-base-multilingual-cased-ner-spanish", "folder":"alvarobartt/"},
    {"id":3,"name":"anonymizer-beto-cased-flair", "folder":"aymurai/"},
    {"id":4,"name":"flair-ner-spanish-judicial", "folder":"aymurai/"},
    {"id":5,"name":"wikineural-multilingual-ner", "folder":"Babelscape/"},
    {"id":6,"name":"bertin-base-ner-conll2002-es", "folder":"bertin-project/"},
    {"id":7,"name":"roberta_model_for_anonimization", "folder":"BSC-LT/"},
    {"id":8,"name":"roberta-base-bne-capitel-ner-plus", "folder":"BSC-LT/"},
    {"id":9,"name":"NER-MEDDOCAN", "folder":"Dnidof/"},
    {"id":10,"name":"bert-base-multilingual-cased-fine_tuned-ner-WikiNeural_Multilingual", "folder":"DunnBC22/"},
    {"id":11,"name":"xlm-roberta-large-finetuned-conll03-english", "folder":"FacebookAI/"},
    {"id":12,"name":"xlm-roberta-large-finetuned-conll03-german", "folder":"FacebookAI/"},
    {"id":13,"name":"ner-multi", "folder":"flair/"},
    {"id":14,"name":"ner-multi-fast", "folder":"flair/"},
    {"id":15,"name":"ner-spanish-large", "folder":"flair/"},
    {"id":16,"name":"AuthorParserModel", "folder":"GEOcite/"},
    {"id":17,"name":"distilbert-base-multilingual-cased-finetuned-conll2003-ner", "folder":"gunghio/"},
    {"id":18,"name":"xlm-roberta-base-finetuned-panx-ner", "folder":"gunghio/"},
    {"id":19,"name":"BETO-finetuned-ner-3", "folder":"ifis/"},
    {"id":20,"name":"span-marker-bert-base-multilingual-cased-multinerd", "folder":"lxyuan/"},
    {"id":21,"name":"span-marker-bert-base-multilingual-uncased-multinerd", "folder":"lxyuan/"},
    {"id":22,"name":"xlm-roberta-large-ner-spanish", "folder":"MMG/"},
    {"id":23,"name":"bert-spanish-cased-finetuned-ner", "folder":"mrm8488/"},
    {"id":24,"name":"RuPERTa-base-finetuned-ner", "folder":"mrm8488/"},
    {"id":25,"name":"TinyBERT-spanish-uncased-finetuned-ner", "folder":"mrm8488/"},
    {"id":26,"name":"NER-fine-tuned-BETO", "folder":"NazaGara/"},
    {"id":27,"name":"flair-ner-multi", "folder":"Omnifact/"},
    {"id":28,"name":"ca_anonimization_core_lg", "folder":"PlanTL-GOB-ES/"},
    {"id":29,"name":"es_anonimization_core_lg", "folder":"PlanTL-GOB-ES/"},
    {"id":30,"name":"roberta-base-bne-capitel-ner", "folder":"PlanTL-GOB-ES/"},
    {"id":31,"name":"roberta-base-bne-capitel-ner-plus", "folder":"PlanTL-GOB-ES/"},
    {"id":32,"name":"roberta-large-bne-capitel-ner", "folder":"PlanTL-GOB-ES/"},
    {"id":33,"name":"DEBERTA_CIEL", "folder":"projecte-aina/"},
    {"id":34,"name":"codeswitch-spaeng-ner-lince", "folder":"sagorsarker/"},
    {"id":35,"name":"span-marker-bert-base-conll2002-es", "folder":"sepulm01/"},
    {"id":36,"name":"es_core_news_lg", "folder":"spacy/"},
    {"id":37,"name":"es_core_news_md", "folder":"spacy/"},
    {"id":38,"name":"es_core_news_sm", "folder":"spacy/"},
    {"id":39,"name":"stanza-es", "folder":"stanfordnlp/"},
    {"id":40,"name":"gliner_multi_pii-v1", "folder":"urchade/"},
    ]
  return models

def select_model(model_index):
  models = get_models()
  if model_index > len(models):
    return None
  else:
    model = models[model_index-1].get("folder") + models[model_index-1].get("name")
    return model

def get_ner_pipe(model):
  ner_pipe = SpanMarkerModel.from_pretrained(model)
  return ner_pipe

def get_ner_pipe_flair(model):
  ner_pipe = SequenceTagger.load(model)
  return ner_pipe

def get_ner_pipe_spacy_md(model):
  ner_pipe = spacy.load("es_core_news_md")
  ner_pipe = es_core_news_md.load()
  return ner_pipe

def get_ner_pipe_spacy_lg_a(model):
  ner_pipe = spacy.load("ca_anonimization_core_lg")
  ner_pipe = ca_anonimization_core_lg.load()
  return ner_pipe

def get_ner_pipe_spacy_sm(model):
  ner_pipe = spacy.load("es_core_news_sm")
  ner_pipe = es_core_news_sm.load()
  return ner_pipe

def get_ner_pipe_spacy_lg(model):
  ner_pipe = spacy.load("es_core_news_lg")
  ner_pipe = es_core_news_lg.load()
  return ner_pipe

def get_ner_pipe_stanza(model):
  ner_pipe = stanza.Pipeline("es")
  return ner_pipe

def get_ner_pipe_urchade(model):
  ner_pipe = GLiNER.from_pretrained(model)
  return ner_pipe


In [ ]:
file = open("tag.json","r")
tag = json.load(file)
file.close()
def get_tag(entity):
  tags=["LOC","PER","ORG", "OTH", "MISC"]
  for tag_l in tags:
    if tag_l in entity:
      return tag[tag_l]
  return "NEW TAG "

In [ ]:
def change_start_end(list_values, n_string):
  for entity in list_values:
    entity["start"] = entity.get("start") + n_string
    entity["end"] = entity.get("end") + n_string
  return list_values

def build_file(entity_list, file_name, min_score = 0.5):
  t = 0
  file= open(file_name,"w")
  end_last = -1
  entity_last = ""
  word_last = ""
  for entity in entity_list:
      if entity['score'] > min_score:
        if (entity['start'] == end_last or end_last == -1) or ((entity['start'] == (end_last+1))
        and get_tag(entity['entity']) == get_tag(entity_last)):
          if (entity['start'] == end_last or end_last == -1):
            word_last += entity['word']
          else:
            word_last += ' ' + entity['word']
          if end_last == -1:
            start_first = entity['start']
            entity_first = entity['entity']
        else:
          t += 1
          end_now = entity['end']
          entity_now = get_tag(entity_first)
          row = 'T'+str(t) +'\t'+ str(entity_now) +' ' + str(start_first) +' '+ str(end_last) +'\t' + word_last.replace('Ġ',' ').replace('Ã±','ñ').replace('Ã¡','á').replace('Ã©','é').replace('ÃŃ','í').replace('Ã³','ó').replace('Ãº','ú').replace('▁',' ').replace('##','') +'\n'
          if(entity_now!='OTHER'):
            file.write(row)
          word_last = entity['word']
          start_first = entity['start']
          entity_first = entity['entity']
        end_last = entity['end']
        entity_last = entity['entity']
  t += 1
  entity_now = get_tag(entity_first)
  row = 'T'+str(t) +'\t'+ str(entity_now) +' ' + str(start_first) +' '+ str(end_last) +'\t' + word_last.replace('Ġ',' ').replace('Ã±','ñ').replace('Ã¡','á').replace('Ã©','é').replace('ÃŃ','í').replace('Ã³','ó').replace('Ãº','ú').replace('▁',' ').replace('##','') +'\n'
  if(entity_now!='OTHER'):
    file.write(row)
  file.close ()

def get_values_512(text, ner_pipe, model_index):
  entity_list = []
  model = select_model(model_index)
  if model != None:
    valors = ner_pipe.predict(text)
    entity_list.extend(valors)
  return get_labels_built(entity_list)

def get_values_512_flair(text, ner_pipe, model_index):
  entity_list = []
  model = select_model(model_index)
  if model != None:
    sentence = Sentence(text)
    ner_pipe.predict(sentence)
  return get_labels_built_flair(sentence)

def get_values_512_spacy(text, ner_pipe, model_index):
  entity_list = []
  model = select_model(model_index)
  if model != None:
    doc = ner_pipe(text)
    valors = ner_pipe.get_pipe("ner")
    aner = valors(doc)
  return get_labels_built_spacy(aner)

def get_values_512_stanza(text, ner_pipe, model_index):
  entity_list = []
  model = select_model(model_index)
  if model != None:
    doc = ner_pipe(text)
  return get_labels_built_stanza(doc)

def get_values_512_urchade(text, ner_pipe, model_index):
  entity_list = []
  model = select_model(model_index)
  if model != None:
    labels = ["work", "booking number", "personally identifiable information", "driver licence", "person", "book", "full address", "company", "actor", "character", "email", "passport number", "Social Security Number", "phone number"]
    chunk_size = 512
    ner_results = []
    for i in range(0, len(text), chunk_size):
      chunk = text[i: i + chunk_size]
      valors = ner_pipe.predict_entities(chunk,labels)
      ner_results.extend(valors)
  return get_labels_built_urchade(ner_results)

def get_labels_built_urchade(entities):
  entities_list = []
  for uno in entities:
    dictionary = {}
    if uno['label'] == "person":
      dictionary['entity'] = 'PER'
    elif uno['label'] == "company":
      dictionary['entity'] = 'ORG'
    elif uno['label'] == "full address":
      dictionary['entity'] = 'LOC'
    else:
      dictionary['entity'] = 'MISC'
    dictionary['score'] = uno['score']
    dictionary['index'] = 1
    dictionary['word'] = uno['text']
    dictionary['start'] = uno['start']
    dictionary['end'] = uno['end']
    entities_list.append(dictionary)
  return entities_list

def get_labels_built_stanza(entities):
  entities_list = []
  for uno in entities.ents:
    dictionary = {}
    dictionary['entity'] = uno.type
    dictionary['score'] = 1
    dictionary['index'] = 1
    dictionary['word'] = uno.text
    dictionary['start'] = uno.start_char
    dictionary['end'] = uno.end_char
    entities_list.append(dictionary)

  return entities_list

def get_labels_built_spacy(sentence):
  entity_list = []

  for ent in sentence.ents:
    dictionary = {}
    dictionary['entity'] = ent.label_
    dictionary['score'] = 1
    dictionary['index'] = 1
    dictionary['word'] = ent.text
    dictionary['start'] = ent.start_char
    dictionary['end'] = ent.end_char
    entity_list.append(dictionary)
  return entity_list

def get_labels_built_flair(sentence):
  entity_list = []
  for entity in sentence.get_spans('ner'):
    dictionary = {}
    dictionary['entity'] = entity.tag
    dictionary['score'] = entity.score
    dictionary['index'] = 1
    dictionary['word'] = entity.text
    dictionary['start'] = entity.start_position
    dictionary['end'] = entity.end_position
    entity_list.append(dictionary)
  return entity_list

def get_labels_built(entities):
  entities_list = []
  for uno in entities:
    dictionary = {}
    dictionary['entity'] = uno['label']
    dictionary['score'] = uno['score']
    dictionary['index'] = 1
    dictionary['word'] = uno['span']
    dictionary['start'] = uno['char_start_index']
    dictionary['end'] = uno['char_end_index']
    entities_list.append(dictionary)
  return entities_list

def prepare_file(text, file_name, folder_system, min_score,
                 type_text, ner_pipe, model_index):

  file_ann_name = folder_system + file_name + ".ann"

  entity_list = []
  if type_text == 2:
    #entity_list = get_values_512(text, ner_pipe = ner_pipe,
                                 #model_index=model_index) # By Span Marker, use on that models type
    #entity_list = get_values_512_flair(text, ner_pipe = ner_pipe,
    #                             model_index=model_index) # By flair, use on that models type

    #entity_list = get_values_512_spacy(text, ner_pipe = ner_pipe,
    #                             model_index=model_index) # By spacy, use on that models type
    #entity_list = get_values_512_stanza(text, ner_pipe = ner_pipe,
    #                             model_index=model_index) # By stanza, use on that models type
    entity_list = get_values_512_urchade(text, ner_pipe = ner_pipe,
                                 model_index=model_index) # By stanza, use on that models type
  if len(entity_list) > 0:
    build_file(entity_list,file_ann_name, min_score = min_score)

def builder(min_score, quantity = 1, type_text = 0, model_index = 1, model_type = 1):
  model = select_model(model_index)
  if model != None:
    #ner_pipe = get_ner_pipe
    #ner_pipe = get_ner_pipe_flair(model)
    #ner_pipe = get_ner_pipe_spacy_lg(model)
    #ner_pipe = get_ner_pipe_spacy_md(model)
    #ner_pipe = get_ner_pipe_spacy_sm(model)
    #ner_pipe = get_ner_pipe_stanza(model)
    ner_pipe = get_ner_pipe_urchade(model)

    folder_base = "system/"
    folder_system = folder_base+"model_"+str(model_index)+"/test"+str(type_text)+"/"
    if not Path(folder_system).exists():
      path = Path(folder_system)
      path.mkdir(parents=True)
    for text in dataset['test']:
        try:
          path_completo = str(text.get('file_name_path'))
          path_completo = path_completo[:-4]

          prepare_file(text.get('text'), text.get('file'), folder_system,
                      min_score = min_score, type_text = type_text,
                      ner_pipe = ner_pipe, model_index = model_index)
          shutil.copy(path_completo+".txt", folder_system+text.get('file')+".txt")
        except:
          print(f"NO PROCESADO   -   ----   --   {text.get('file')}")
        quantity -= 1
        print(quantity)
        if quantity < 1:
          break

def builder_all():
  for i in range(0,4):
    for j in range(1,25):
      builder(type_text = i, min_score = 0.0, quantity = 1, model_index = j)

def builder_all_test(start_model = 1, end_model = 1, text = 2, quantity_test = 1):
  for number_model in range(start_model,end_model):
    builder(min_score = 0.0, quantity = quantity_test, type_text = text, model_index = number_model)

In [ ]:
builder_all_test(start_model = 40, end_model = 41, text = 2, quantity_test = 250)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [ ]:
def comprimir():
  fantasy_zip = zipfile.ZipFile('/content/system.zip', 'w')
  for folder, subfolders, filesFolder in os.walk('/content/system'):
      for file in filesFolder:
          if file.endswith('.conll') or file.endswith('.txt') or file.endswith('.ann'):
              fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content/system'), compress_type = zipfile.ZIP_DEFLATED)
  fantasy_zip.close()

In [ ]:
comprimir()